In [3]:
# @title Just need to click the "run/play" button, select from the available options by typing the number and them enter
# Clone the repository (if not already cloned)
!git clone https://github.com/lopezbec/MASt3R_Laf_DEMO.git

# Install required libraries
!pip install trimesh plotly

import os
import trimesh
import plotly.graph_objects as go
import numpy as np
clear_output()

# Define the directory containing the .ply files in the cloned repo
ply_dir = "MASt3R_Laf_DEMO/ply_files"

# List all .ply files in that directory
ply_files = [f for f in os.listdir(ply_dir) if f.endswith(".ply")]
if not ply_files:
    raise ValueError("No .ply files found in the directory: " + ply_dir)

print("Available .ply files:")
for idx, f in enumerate(ply_files):
    print(f"{idx}: {f}")

# Prompt user to select a file by entering its index
selected_index = input("Enter the index of the file you want to render: ")

try:
    selected_index = int(selected_index)
    selected_filename = ply_files[selected_index]
    selected_file = os.path.join(ply_dir, selected_filename)
    print("Selected file:", selected_file)
except Exception as e:
    raise ValueError("Invalid selection. Please enter a valid index.") from e

# Load the selected .ply file using trimesh
mesh = trimesh.load(selected_file)

# Check if vertex colors exist (expecting an (N,3) or (N,4) array)
vertex_colors = None
if hasattr(mesh.visual, 'vertex_colors') and mesh.visual.vertex_colors is not None:
    vcolors = mesh.visual.vertex_colors
    if vcolors.shape[1] == 4:  # Drop alpha channel if present
        vcolors = vcolors[:, :3]
    vertex_colors = ["rgb({},{},{})".format(int(r), int(g), int(b)) for r, g, b in vcolors]

# Depending on whether the file contains face data (mesh) or just points, choose the Plotly object.
if hasattr(mesh, "faces") and mesh.faces is not None and len(mesh.faces) > 0:
    vertices = mesh.vertices
    faces = mesh.faces
    if vertex_colors is not None:
        fig = go.Figure(data=[go.Mesh3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            i=faces[:, 0],
            j=faces[:, 1],
            k=faces[:, 2],
            vertexcolor=vertex_colors,
            opacity=1.0
        )])
    else:
        fig = go.Figure(data=[go.Mesh3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            i=faces[:, 0],
            j=faces[:, 1],
            k=faces[:, 2],
            color='lightblue',
            opacity=1.0
        )])
else:
    # Fallback: plot as a point cloud
    vertices = mesh.vertices
    if vertex_colors is not None:
        fig = go.Figure(data=[go.Scatter3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            mode='markers',
            marker=dict(size=2, color=vertex_colors)
        )])
    else:
        fig = go.Figure(data=[go.Scatter3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            mode='markers',
            marker=dict(size=2, color='blue')
        )])

# Remove axes and background for a cleaner view
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False, showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(visible=False, showticklabels=False, showgrid=False, zeroline=False),
        zaxis=dict(visible=False, showticklabels=False, showgrid=False, zeroline=False),
        bgcolor='rgba(0,0,0,0)'  # Transparent scene background
    ),
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent overall background
    margin=dict(l=0, r=0, t=0, b=0)  # Remove margins
)

fig.show()


Available .ply files:
0: small_copy.ply
1: small.ply


KeyboardInterrupt: Interrupted by user